In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time

def download_files(url, download_path):
    # Send a request to the website
    session = requests.Session()
    # Set a user-agent for the session
    session.headers.update(
        {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'})

    # Send a request to the website
    response = session.get(url)
    response.raise_for_status()

    # Create a BeautifulSoup object
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all the 'a' tags with text 'Скачать PDF'
    pdf_links = soup.find_all('a', string='Скачать PDF')

    # Loop through the 'a' tags to download the files
    for link in pdf_links:
        # Get the URL
        gdrive_url = link['href']
        # Extract the file ID from the URL
        file_id = gdrive_url.split('id=')[1].split('&')[0]
        # Try to find the closest preceding header tag for the original filename
        original_filename_div = link.find_previous('div', class_='t-card__title t-name t-name_lg')
        if original_filename_div:
            original_filename = original_filename_div.text
        else:
            # If no div is found, use the file ID as the filename
            original_filename = file_id
        
        # Replace any characters in filename that are not allowed in file names
        original_filename = original_filename.replace('/', '-').replace(':', '-')
        
        # Construct the direct download link
        direct_download_link = f'https://drive.google.com/uc?export=download&id={file_id}'
        
        # Download the file
        response = requests.get(direct_download_link)
        response.raise_for_status()  # Check for a valid response
        # Write the file to disk
        with open(os.path.join(download_path, f'{original_filename}.pdf'), 'wb') as f:
            f.write(response.content)
        print(f'Downloaded {original_filename}.pdf')
        time.sleep(1)

# Specify the URL of the webpage and the path where you want to download the files
url = 'https://roag-portal.ru/recommendations_obstetrics'
download_path = 'akusher_pdfs'

# Call the function to download the files
download_files(url, download_path)